## Interactuando con Github

In [6]:
from github import Github
import os
from dotenv import load_dotenv, find_dotenv
from time import sleep
import re
import unicodedata
import json
import pickle

load_dotenv(find_dotenv())


git_key = os.getenv("GITHUB_ADMIN_TOKEN")
g = Github(git_key)

def download_github_data():
    with open("github_admin/files/github_books.pkl", "wb") as file:
        books = _get_books()
        pickle.dump(books, file)


def load_github_data():
    books = []
    with open("files/github_books.pkl", "rb") as file:
        try:
            while True:
                books.append(pickle.load(file))
        except EOFError:
            return books[0]
        
def _get_books():
    """Comprueba si un repositorio tiene archivo SUMMARY.md, indicativo de que
    se trata de un Gitbook.

    Devuelve:
        listado de gitbook de la organización CATEDU.
    """
    global g
    repos = list(g.get_organization("catedu").get_repos())
    gitbooks = [repo for repo in repos if _has_summary(repo)]
    return gitbooks

def _has_summary(repo):
    """Comprueba si un repositorio tiene archivo SUMMARY.md, indicativo de que
    se trata de un Gitbook.

    Parámetros:
        repo (Objeto PyGithub que representa un repositorio)

    Devuelve:
        True or False (Boolean) indicando si tiene o no SUMMARY.md
    """
    try:
        contents = [content.name for content in repo.get_contents("")]
        it_has_summary = "SUMMARY.md" in contents
        return it_has_summary
    except:
        print(f"######### El repo {repo.name} falla #############")
        
def repos_data():
    books = load_github_data()
    data = [
        {
            "repo_name": book.name,
            "master_url": book.html_url,
            "gh-pages": f"https://catedu.github.io/{book.name}/",
        }
        for book in books
    ]
    return data

In [12]:
download_github_data()

In [7]:
books = load_github_data()

In [20]:
",".join([book.name for book in books])

'abies-lectores,monta-tu-aula-de-informatica-con-raspberry-pi,la-ensenanza-aprendizaje-de-la-flauta-escolar,google_classroom,curso_de_edmodo,faq-aularagon,curso_linux,matematicas-tic,sintaxis_con_las_tic,procedimiento-administrativo,curso-de-tutores,manual-de-creadores,metodologias-activas-didactica-y-evaluacion,radio,app-inventor-course,abies-administrador,video-tutoriales,ensena-pensamiento-computacional-con-arduino,ensena-pensamiento-computacional-con-wego-wedo,ensena-pensamiento-computacional-con-scratch,scratch-avanzado-y-makey-makey,introduccion-a-python,libreOffice-la-suite-ofimatica-libre,libreoffice-base-curso,libreoffice-writer-curso,libreoffice-calc-curso,libreoffice-impress-curso,libreoffice-draw-math,programa-arduino-mediante-codigo,robotica-educativa-con-mbot,robotica-educativa-con-mbot-II,robotica-en-infantil-con-bee-bot,scribus-maqueta-visualmente-tus-contenidos-educativos,aprendizaje-colaborativo-con-blog,modulo-5-asesores,atrevete-con-wordpress,programa-arduino-con-ec

In [8]:
contents = books[0].get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(books[0].get_contents(file_content.path))
    else:
        print(file_content)

ContentFile(path=".gitignore")
ContentFile(path=".nojekyll")
ContentFile(path="FOOTER.md")
ContentFile(path="LICENSE")
ContentFile(path="README.md")
ContentFile(path="SUMMARY.md")
ContentFile(path="book.json")
ContentFile(path="consultar_catlogos_de_otras_bibliotecas.md")
ContentFile(path="consultar_el_catlogo_de_fondos_de_la_biblioteca_de_mi_centro.md")
ContentFile(path="crditos.md")
ContentFile(path="darme_de_alta_en_abiesweb_para_usar_la_biblioteca_de_mi_centro_escolar.md")
ContentFile(path="escribir_una_opinin_sobre_algn_fondo_de_los_que_hemos_tomado_en_prstamo.md")
ContentFile(path="modificar_mi_contrasea_de_acceso.md")
ContentFile(path="realizar_reservas_de_ejemplares.md")
ContentFile(path="seguir_las_noticias_de_la_biblioteca_de_mi_centro.md")
ContentFile(path="solicitar_una_prrroga_del_prstamo_actual.md")
ContentFile(path="img/Boletin.JPG")
ContentFile(path="img/BusquedaEjemplaresLEctor.JPG")
ContentFile(path="img/BusquedaEnRed.JPG")
ContentFile(path="img/ListadoCatalogoLector.